<a href="https://colab.research.google.com/github/Richardxiao123/TW_stock_analysis/blob/main/%E5%9B%9E%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#載入所有股票代號

In [ ]:
import pandas as pd

stock_code =pd.read_csv("stock_code.csv")
stock_code=stock_code.drop(columns=['stock_code'])
stock_code = stock_code.reset_index().rename(columns={'index': 'stockprice'})

print(stock_code)




     stockprice
0          1101
1         1101B
2          1102
3          1103
4          1104
...         ...
1068       9944
1069       9945
1070       9946
1071       9955
1072       9958

[1073 rows x 1 columns]


# 數據處理


In [ ]:
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

data_test = pd.DataFrame()


for a in stock_code['stockprice']:
    ticker =f"{a}.TW"
    data = yf.download(ticker,start='2024-2-19', end="2024-3-7")

    if data.empty:
        print(f"{ticker}.無資料跳過")
        continue

    # 進行數據標準化
    scaler = MinMaxScaler()
    data_temp = pd.DataFrame(index=data.index)
    data_temp['name'] = f"{a}.TW"

    for x in ['Close', 'High', 'Low', 'Open', 'Volume']:
        data_temp[x] = scaler.fit_transform(data[[x]])  # 線性變換 0~1

    # 10 天為一組轉到同一排
    new_data = {}
    new_data['name'] = [data_temp['name'].iloc[0]]

    for i in ['Close', 'High', 'Low', 'Open']:
        for y in range(10):
            new_data[f"{i}{y+1}"] = data_temp[i].iloc[y]

    new_data = pd.DataFrame(new_data)
    data_test = pd.concat([data_test, new_data], ignore_index=True)
    #print(new_data)

print(data_test)


In [ ]:
from google.colab import files
data_test.to_csv('data_run.csv')
files.download('data_run.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 開始預測

In [ ]:
import pickle
import pandas as pd

with open("rf_model.pkl", "rb") as f:
    rf_model = pickle.load(f)

#讀檔
data_run =pd.read_csv("data_run.csv")

X_test=data_run.drop(columns=['name']).drop(columns=['Unnamed: 0'])
name=data_run['name']
print(X_test)

y_pred = rf_model.predict(X_test)

print(y_pred)

count=0
buy_list=[]
for n in y_pred:
    if n == 5:
        print(name[count])
        buy_list.append(name[count])
        count+=1
    else:
          count+=1

print(buy_list)

       Close1    Close2    Close3    Close4    Close5    Close6    Close7  \
0    1.000000  0.842106  0.684211  0.736840  0.578949  0.315789  0.000000   
1    0.624999  0.749993  0.749993  0.749993  0.000000  0.249998  0.749993   
2    0.333330  0.444446  0.666661  0.888893  0.444446  0.555554  0.222214   
3    0.636363  0.909090  0.909090  1.000000  0.818180  1.000000  0.545457   
4    0.108695  0.000000  0.021740  0.086956  0.130435  0.217391  0.152173   
..        ...       ...       ...       ...       ...       ...       ...   
951  0.596154  0.365384  0.288462  0.288462  0.269231  0.115385  0.000000   
952  0.400010  0.400010  0.199997  0.400010  0.000000  0.199997  0.400010   
953  0.000000  0.071429  0.142857  0.190476  0.452381  0.690476  0.738095   
954  1.000000  0.968750  0.984375  0.937500  0.875001  0.640626  0.531250   
955  0.128571  0.100000  0.000000  0.242857  0.028572  0.085714  0.142857   

       Close8    Close9   Close10  ...     Open1         Open2     Open3  \

# 回測數據

In [ ]:
tickers=['1615.TW','2330.TW']
for ticker in tickers:
  data = yf.download(ticker,start='2024-3-6', end="2024-3-7")

  print(data)


ERROR:yfinance:Failed to get ticker '1615.TW' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['1615.TW']: YFTzMissingError('possibly delisted; no timezone found')


Empty DataFrame
Columns: [(Adj Close, 1615.TW), (Close, 1615.TW), (High, 1615.TW), (Low, 1615.TW), (Open, 1615.TW), (Volume, 1615.TW)]
Index: []


[*********************100%***********************]  1 of 1 completed

Price          Close        High         Low        Open    Volume
Ticker       2330.TW     2330.TW     2330.TW     2330.TW   2330.TW
Date                                                              
2024-03-06  722.7547  725.704719  705.054585  706.037924  43687122


In [ ]:
import pandas as pd
import numpy as np


def augment_data(data: pd.DataFrame, target_size: int, noise_level: float = 0.02) -> pd.DataFrame:

    num_new_samples = target_size - len(data)
    np.random.seed(0)

    # 隨機從原始資料中選擇資料來進行增強
    selected_indices = np.random.choice(data.index, num_new_samples, replace=True)
    new_samples = data.iloc[selected_indices].copy()

    # 對數據進行增強
    for col in data.columns[3:]:  # 忽略 'Unnamed: 0', 'unnamed', 'score'
        original_values = new_samples[col].values
        noise = np.random.normal(0, noise_level, original_values.shape)

        # 隨機選擇 Scaling 或 Shifting
        random_choice = np.random.rand(len(original_values))
        adjusted_values = np.where(
            random_choice < 0.5,
            original_values * (1 + noise),  # 微調 (Scaling)
            original_values + noise  # 加入噪音 (Shifting)
        )

        new_samples[col] = adjusted_values

    # 將原始資料與增強資料合併
    augmented_data = pd.concat([data, new_samples], ignore_index=True)
    return augmented_data